### Step 2: Perform Name Entity Recognition (NER) using [NameTag 3](https://lindat.mff.cuni.cz/services/nametag/) service

Inputs:

- `GALAXY_INPUTS["cleanTextDataset"]` - A JSONL Dataset with clean text parsed from the web source`

Outputs:

- `outputs/2_pausanias_nametag.jsonl` - A JSONL Dataset with the results of the calls to NameTag 3

In [ ]:
# Only run on Galaxy
!pip install tqdm requests beautifulsoup4 srsly

In [ ]:
import tqdm
import requests
import srsly

In [ ]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36"}

In [ ]:
data = list(srsly.read_jsonl(GALAXY_INPUTS["cleanTextDataset"][0]["path"]))

In [ ]:
for element in tqdm.tqdm(data):
    x = requests.get(f"https://lindat.mff.cuni.cz/services/nametag/api/recognize?data={element.get("text")}&model=nametag3-multilingual-conll-250203&output=vertical", headers=headers).json()
    new_element = [i.split("\t")[-1].replace("\xad","") for i in x.get("result").splitlines() if i.split("\t")[1] != "PER"]

    element.update({"nametag_mentions":new_element})

    srsly.write_jsonl("outputs/2_pausanias_nametag.jsonl", [element], append=True, append_new_line=False)